In [41]:
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow_datasets as tfds
from   tensorflow.keras.datasets import fashion_mnist as fmnist
from   tensorflow.keras.layers import (
    Conv2D as Conv, MaxPooling2D as Pool, Dense, Dropout, Flatten)
from   tensorflow.keras.models import Sequential
from   tensorflow.nn import relu, softmax

In [2]:
mnist_data = tfds.load('fashion_mnist')
for item in mnist_data:
    print(item)

train
test


In [3]:
mnist_train = tfds.load(name='fashion_mnist', split='train')

In [4]:
isinstance(mnist_train, tf.data.Dataset)

True

In [5]:
isinstance(mnist_data, tf.data.Dataset)

False

In [6]:
for item in mnist_train.take(1):
    print(type(item))
    print(item.keys())

<class 'dict'>
dict_keys(['image', 'label'])


In [7]:
for item in mnist_train.take(1):
    print(item['image'].shape)
    print(item['label'])

(28, 28, 1)
tf.Tensor(2, shape=(), dtype=int64)


In [8]:
mnist_test, info = tfds.load(name='fashion_mnist', with_info=True)
print(info)

tfds.core.DatasetInfo(
    name='fashion_mnist',
    full_name='fashion_mnist/3.0.1',
    description="""
    Fashion-MNIST is a dataset of Zalando's article images consisting of a training set of 60,000 examples and a test set of 10,000 examples. Each example is a 28x28 grayscale image, associated with a label from 10 classes.
    """,
    homepage='https://github.com/zalandoresearch/fashion-mnist',
    data_path='/Users/dsatterthwaite/tensorflow_datasets/fashion_mnist/3.0.1',
    download_size=29.45 MiB,
    dataset_size=36.42 MiB,
    features=FeaturesDict({
        'image': Image(shape=(28, 28, 1), dtype=tf.uint8),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
    }),
    supervised_keys=('image', 'label'),
    splits={
        'test': <SplitInfo num_examples=10000, num_shards=1>,
        'train': <SplitInfo num_examples=60000, num_shards=1>,
    },
    citation="""@article{DBLP:journals/corr/abs-1708-07747,
      author    = {Han Xiao and
                 

In [18]:
(X_train, y_train), (X_test, y_test) = fmnist.load_data()

In [19]:
(X_train, y_train), (X_test, y_test) = tfds.as_numpy(
    tfds.load('fashion_mnist', 
              split=['train', 'test'], 
              batch_size=-1, 
              as_supervised=True))

In [20]:
X_train = X_train / 255.
X_test = X_test / 255.

In [23]:
mod = Sequential([Flatten(input_shape=(28, 28, 1)),
                  Dense(128, activation=relu),
                  Dropout(0.2),
                  Dense(10, activation=softmax)])

In [24]:
mod.compile(optimizer='adam', 
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy'])

In [25]:
mod.fit(X_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 3s 1ms/step - loss: 0.6742 - accuracy: 0.7613
Epoch 2/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.4012 - accuracy: 0.8553
Epoch 3/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.3664 - accuracy: 0.8679: 0s - l
Epoch 4/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.3438 - accuracy: 0.8729: 0s - loss: 0.3435 - accu
Epoch 5/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.3309 - accuracy: 0.8782: 0s - loss: 0.3


In [26]:
data = tfds.load('horses_or_humans', split='train', as_supervised=True)
train_batches = data.shuffle(100).batch(100)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/1027 [00:00<?, ? examples/s]

Shuffling horses_or_humans-train.tfrecord...:   0%|          | 0/1027 [00:00<?, ? examples/s]

Generating test examples...:   0%|          | 0/256 [00:00<?, ? examples/s]

Shuffling horses_or_humans-test.tfrecord...:   0%|          | 0/256 [00:00<?, ? examples/s]

Dataset horses_or_humans downloaded and prepared to /Users/dsatterthwaite/tensorflow_datasets/horses_or_humans/3.0.0. Subsequent calls will reuse this data.


In [31]:
CKERN = (3, 3)
PKERN = (2, 2)
mod = Sequential([
    Conv(16, CKERN, activation='relu', input_shape=(300, 300, 3)),
    Pool(PKERN), 
    Conv(32, CKERN, activation='relu'),
    Pool(PKERN),
    Conv(64, CKERN, activation='relu'),
    Pool(PKERN),
    Conv(64, CKERN, activation='relu'),
    Pool(PKERN),
    Conv(64, CKERN, activation='relu'),
    Pool(PKERN),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(1, activation='sigmoid')])

In [32]:
mod.compile(
    optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])

In [33]:
hist = mod.fit(train_batches, epochs=5)

Epoch 1/5
11/11 [==============================] - 32s 3s/step - loss: 29.2058 - accuracy: 0.5419
Epoch 2/5
11/11 [==============================] - 29s 3s/step - loss: 0.3382 - accuracy: 0.8622
Epoch 3/5
11/11 [==============================] - 30s 3s/step - loss: 0.1693 - accuracy: 0.9347
Epoch 4/5
11/11 [==============================] - 29s 3s/step - loss: 0.0795 - accuracy: 0.9657
Epoch 5/5
11/11 [==============================] - 29s 3s/step - loss: 0.0345 - accuracy: 0.9902


### Adding image augmentation

In [38]:
def augment(img, label):
    img = tf.cast(img, tf.float32)
    img = img / 255
    img = tf.image.random_flip_left_right(img)
    return img, label

In [40]:
train = data.map(augment)
train_batches = train.shuffle(100).batch(100)

### Addons

In [43]:
def augment(img, label):
    img = tf.cast(img, tf.float32)
    img = img / 255
    img = tf.image.random_flip_left_right(img)
    img = tfa.image.rotate(img, 40, interpolation='NEAREST')
    return img, label